# Experiment 06: Factoid Subsample — Condition Examples

This notebook shows the actual text for each experimental condition using real data from the dataset. No GPU needed.

In [ ]:
import os, sys, json, re
import numpy as np
from pathlib import Path
from collections import Counter

sys.path.insert(0, "../..")
from lib.data import count_words

SEED = 43  # Different seed from main experiments!

from datasets import load_dataset
ds = load_dataset("microsoft/ms_marco", "v1.1", split="validation")

samples = []
for item in ds:
    if len(samples) >= 1500:
        break
    passages = item.get('passages', {})
    ptexts = passages.get('passage_text', [])
    is_sel = passages.get('is_selected', [])
    query = item.get('query', '')
    answers = item.get('answers', [])
    well_formed = item.get('wellFormedAnswers', [])
    answer = None
    if well_formed and len(well_formed) > 0 and well_formed[0] not in ('[]', ''):
        answer = well_formed[0]
    elif answers and len(answers) > 0 and answers[0] != 'No Answer Present.':
        answer = answers[0]
    if not answer or len(answer.split()) > 5:
        continue  # FACTOID FILTER: answer <= 5 words
    for pt, sel in zip(ptexts, is_sel):
        wc = count_words(pt)
        if sel == 1 and 30 <= wc <= 300:
            samples.append({
                'passage': pt, 'query': query, 'answer': answer,
                'word_count': wc, 'answer_words': len(answer.split()),
            })
            break

np.random.seed(SEED)
np.random.shuffle(samples)
samples = samples[:500]
del ds

STOP_WORDS = {
    'a', 'an', 'the', 'is', 'are', 'was', 'were', 'be', 'been', 'being',
    'have', 'has', 'had', 'do', 'does', 'did', 'will', 'would', 'could',
    'should', 'may', 'might', 'can', 'shall', 'to', 'of', 'in', 'for',
    'on', 'with', 'at', 'by', 'from', 'as', 'into', 'through', 'during',
    'before', 'after', 'above', 'below', 'between', 'and', 'but', 'or',
    'not', 'no', 'if', 'then', 'than', 'so', 'up', 'out', 'about',
    'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those',
    'it', 'its', 'i', 'me', 'my', 'we', 'our', 'you', 'your', 'he',
    'him', 'his', 'she', 'her', 'they', 'them', 'their', 'how', 'when',
    'where', 'why', 'much', 'many', 'some', 'any', 'all', 'each',
    'does', 'also', 'just', 'more', 'most', 'very', 'too', 'only',
}
def extract_keywords(text):
    words = re.sub(r'[^\w\s]', '', text.lower()).split()
    return [w for w in words if w not in STOP_WORDS and len(w) > 2]
def make_surrogate_template(passage):
    content_words = extract_keywords(passage)
    if not content_words:
        return "What is this about?"
    counts = Counter(content_words)
    return f"What is {counts.most_common(1)[0][0]}?"

# Verify
ckpt_path = Path("../../results/exp06/checkpoint.json")
if ckpt_path.exists():
    ckpt = json.loads(ckpt_path.read_text())
    if ckpt['results'][0]['query'][:50] == samples[0]['query'][:50]:
        print("Checkpoint verification: MATCH")

print(f"Loaded {len(samples)} FACTOID samples (answer <= 5 words, SEED={SEED})")
print()

ex = samples[0]


def show_sample(s, doc_key='passage', n=0):
    # Show sample info
    doc = s[doc_key]
    print(f"{'='*80}")
    print(f"SAMPLE {n}")
    print(f"{'='*80}")
    print(f"  Query:    {s['query']}")
    print(f"  Answer:   {s['answer']}")
    print(f"  Document: {doc[:100]}...")
    print(f"  Doc words: {len(doc.split())}")
    print()

def show_conditions(conditions, doc_text):
    # conditions: list of (name, description, encoder_prefix_text_or_None)
    # For bare conditions, encoder_prefix_text is None
    print(f"{'Condition':<30} {'Prefix':<14} {'Encoder input (first 70 chars)'}")
    print(f"{'-'*100}")
    for name, desc, prefix_text in conditions:
        if prefix_text is None:
            enc_preview = doc_text[:70]
            print(f"{name:<30} {'(none)':<14} {enc_preview}...")
        else:
            enc_text = prefix_text + "\n" + doc_text
            print(f"{name:<30} {str(len(prefix_text.split()))+'w':<14} {enc_text[:70]}...")
        if desc:
            print(f"  {'':>28} ^ {desc}")
    print()

print(f"FACTOID SAMPLE (answer <= 5 words):")
show_sample(ex, n=0)
print(f"  Answer length: {ex['answer_words']} words")
print()

query_words = ex['query'].split()
n_qw = len(query_words)
other_idx = (0 + 250) % len(samples)
rand_matched = " ".join(samples[other_idx]['passage'].split()[:n_qw])
rng = np.random.RandomState(SEED + 0)
shuffled = list(query_words)
rng.shuffle(shuffled)
scrambled = " ".join(shuffled)
surr_template = make_surrogate_template(ex['passage'])

conditions = [
    ("bare", "Baseline", None),
    ("oracle_x1_trunc", "Real query x 1", ex['query']),
    ("oracle_x4_trunc", "Real query x 4", " ".join([ex['query']] * 4)),
    ("random_x1_trunc", "Random matched x 1", rand_matched),
    ("random_x4_trunc", "Random matched x 4", " ".join([rand_matched] * 4)),
    ("scrambled_oracle_trunc", "Query words shuffled", scrambled),
    ("surr_template_x1_trunc", "'What is [kw]?' x 1", surr_template),
    ("surr_template_x4_trunc", "'What is [kw]?' x 4", " ".join([surr_template] * 4)),
]
show_conditions(conditions, ex['passage'])

print("KEY QUESTION: Does the two-population structure exist?")
print("  Factoid QA (short answers) might have more semantic headroom.")
print("  Overall 85% structural might be a Simpson's paradox average.")
